In [1]:
# Importación de las librerías requeridas. La librería 'sys' provee variables y funcionalidades relacionadas con el intérprete. La librería 'datetime' y 'time'
# proprorcionan clases para manipular fechas y horas. La librería 'pymongo' ayuda a conectarse a una base de datos MongoBD. La librería 'pandas' facilita la
# manipulación y tratamiento de datos.
import sys
import datetime, time
import pymongo 
import pandas as pd

# Con la librería 'BeautifulSoup' se extrae información de contenido en formato HTML o XML. Con 'selenium' se pueden crear pruebas autimatizadas que permiten
# reproducir el ambiente real de una aplicación.
from bs4 import BeautifulSoup
from selenium import webdriver
from pymongo import MongoClient, TEXT

In [2]:
# En la variable SITE_URL se almacenará el sitio que se analizará.
SITE_URL = 'https://www.elespectador.com'

In [3]:
# La variable DRIVER_PATH tendrá un enlace de conexión al navegador Firefox. GeckoDriver es un proxy que ayuda a comunicarse con los navegadores basados en Gecko.
DRIVER_PATH = './geckodriver.exe'

In [4]:
# Creación de una nueva ventana FireFox.
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\harol\AppData\Local\Temp/ipykernel_1900/4016513992.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [5]:
# 'make_request' va a ir al URL que se le asigne, simular la función de Scroll y detenerse sólo cuando llegue al final de la página.
def make_request(browser, relative_path):
    # Se realiza la solicitud al navegador.
    browser.get(SITE_URL + relative_path)
    
    # Simulación del Scroll vertical.
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Se obtiene el HTML para después pasarlo a BeautifulSoup para el análisis scrapping.
    return BeautifulSoup(browser.page_source, 'html.parser')

> Categoría Judicial

In [6]:
# Conexión a la base de datos 'noticias' y específicamente a una nueva colección llamada 'judicial'.
client = MongoClient('localhost', 27017)
db = client['noticias']
collection = db['judicial']

In [7]:
# Creación una colección de diccionarios y específicación exacta de la sección que se analizará.
news = []
url = '/archivo/judicial/'

# Se hará un recorrido de tan solo 16 páginas de la sección especificada.
for n in range(1, 16):
    # Acceso a la página 1 del sitio, la variable 'n' representa el número de la página; aumentará conforme vaya avanzando el ciclo.
    # Se localiza la clase 'Layout-flexAds' para almacenar la información en la variable layout.
    # Dentro de la clase se anterior (layout) se buscarán las clases llamadas 'section', que son dos bloques, cada uno con 4 noticias.
    # Por último, se busca la clase 'Card' para concatenar toda la información.
    soup = make_request(browser, url + str(n))
    layout = soup.find(class_ = 'Layout-flexAds')
    blocks = layout.find('section').find_all(recursive = False)
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    for card in cards:
        # Por cada tarjeta se va a extraer la etiqueta h2 con la clase 'Card-title', estas etiquetas dentro de un elemento "a".
        # El 'relative_path' nos dirige a la página específica de la noticia.
        news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [8]:
# Después se dirige a la página de cada noticia.
for i in news:
    # Obtener el contenido HTML para cada página de noticias.
    soup = make_request(browser, i['relative_path'])
    
    # Se extrae la fecha, autor y el resumen de la noticia.
    i['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    i['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    i['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Se obtiene el texto completo de la noticia.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    i['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [9]:
news

[{'title': 'Violencia destruyó promesa de paz para muchos colombianos: Consejo Noruego',
  'relative_path': '/colombia/violencia-destruyo-promesa-de-paz-para-muchos-colombianos-consejo-noruego/',
  'datetime': '9 abr 2022  - 1:09 p.\xa0m.',
  'author': 'Agencia EFE',
  'summary': 'La oenegé Consejo Noruego para Refugiados advierte de la escalada violenta que vive el país en la conmemoración del 9 de abril, Día de las Víctimas. ',
  'full_text': 'El aumento del conflicto y la violencia “han destruido la promesa de paz para muchos colombianos”, alertó este viernes el director para el país de la ONG Consejo Noruego para Refugiados (NRC, por sus siglas en inglés), Francesco Volpi. (Lea también: Cinco hitos en los 10 años de conmemoración a víctimas del conflicto) “El Gobierno y los grupos armados deben unirse y ponerse de acuerdo para poner fin a esta pesadilla para los colombianos, muchos de los cuales ya han soportado décadas de un conflicto brutal”, aseguró Volpi en la conmemoración del

In [10]:
collection.insert_many(news)

> Política

In [11]:
client = MongoClient("localhost", 27017)
db = client["noticias"]
collection = db["politica"]

In [12]:
news = []
url = '/archivo/politica/elecciones-colombia-2022/'

for n in range(1, 16):
    soup = make_request(browser, url + str(n))
    layout = soup.find(class_ = 'Layout-Container Layout-flexAds')
    blocks = layout.find('section').find_all(recursive = False)
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    for card in cards:
        news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [13]:
for i in news:
    soup = make_request(browser, i['relative_path'])
    
    i['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    i['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    i['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    paragraphs = soup.find_all(class_ = 'font--secondary')
    i['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [14]:
news

[{'title': 'Las indirectas de Iván Duque a las propuestas de Petro',
  'relative_path': '/politica/elecciones-colombia-2022/en-video-las-indirectas-de-ivan-duque-a-las-propuestas-de-petro/',
  'datetime': '9 abr 2022  - 12:22 p.\xa0m.',
  'author': 'Unidad de Video',
  'summary': 'En su discurso en Boyacá, el primer mandatario lanzó varias pullas en contra de la propuesta minera del candidato del Pacto Histórico, que ha señalado en su plan de gobierno que le pondría un coto a la minería a cielo abierto y revisaría las concesiones.',
  'full_text': ''},
 {'title': 'Sergio Fajardo: un estilo que otros quieren imitar',
  'relative_path': '/politica/elecciones-colombia-2022/sergio-fajardo-un-estilo-que-otros-quieren-imitar/',
  'datetime': '9 abr 2022  - 10:31 a.\xa0m.',
  'author': 'Lucety Carreño Rojas',
  'summary': 'El candidato fue alcalde de Medellín (2004-2007) y gobernador de Antioquia (2012 – 2015). En 2022 se postuló como presidente por la Coalición Centro Esperanza y ganó la con

In [15]:
collection.insert_many(news)

> Educación

In [20]:
client = MongoClient("localhost", 27017)
db = client["noticias"]
collection = db["educacion"]

In [21]:
news = []
url = '/archivo/educacion/'

for n in range(1, 16):
    soup = make_request(browser, url + str(n))
    layout = soup.find(class_ = 'Layout-flexAds')
    blocks = layout.find('section').find_all(recursive = False)
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    for card in cards:
        news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [22]:
for i in news:
    soup = make_request(browser, i['relative_path'])
    
    i['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    i['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    i['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    paragraphs = soup.find_all(class_ = 'font--secondary')
    i['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [23]:
news

[{'title': 'Del 18 al 21 de abril se aplicarán las Pruebas Saber 3°, 5°, 7° y 9°',
  'relative_path': '/educacion/del-18-al-21-de-abril-se-aplicaran-las-pruebas-saber-3-5-7-y-9-noticias-colombia/',
  'datetime': '8 abr 2022  - 11:56 a.\xa0m.',
  'author': 'Redacción Educación',
  'summary': '200.000 estudiantes, de 1.300 sedes educativas seleccionadas en los 32 departamentos del país, deberán realizar la prueba.',
  'full_text': 'El Ministerio de Educación Nacional y el Instituto Colombiano para la Evaluación de la Educación (Icfes) anunciaron que entre el 18 y el 21 de abril se aplicarán las Pruebas Saber para los grados 3°, 5°, 7° y 9°. 200.000 estudiantes, de 1.300 sedes educativas seleccionadas en los 32 departamentos del país, deberán realizar la prueba en esta ocasión.  María Victoria Angulo, ministra de Educación, reconoció el trabajo que han adelantado los rectores y rectoras del país y les hizo un llamado para que las pruebas se hagan presencialmente, bajo la modalidad de pape

In [24]:
collection.insert_many(news)

> Macroeconomía

In [28]:
client = MongoClient("localhost", 27017)
db = client["noticias"]
collection = db["macroeconomia"]

In [31]:
news = []
url = '/archivo/economia/macroeconomia/'

for n in range(1, 15):
    soup = make_request(browser, url + str(n))
    layout = soup.find(class_ = 'Layout-flexAds')
    blocks = layout.find('section').find_all(recursive = False)
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    for card in cards:
        news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [32]:
for i in news:
    soup = make_request(browser, i['relative_path'])
    
    i['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    i['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    i['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    paragraphs = soup.find_all(class_ = 'font--secondary')
    i['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [33]:
news

[{'title': 'La inflación sube en Latinoamérica y también lo seguirán haciendo las tasas',
  'relative_path': '/economia/macroeconomia/la-inflacion-sube-en-latinoamerica-y-tambien-lo-seguiran-haciendo-las-tasas-de-interes/',
  'datetime': '9 abr 2022  - 1:54 p.\xa0m.',
  'author': 'Agencia Bloomberg',
  'summary': 'Los banqueros centrales de América Latina enfrentan una presión renovada para extender los intensos aumentos a las tasas de interés después de que los precios al consumidor superaron las estimaciones en toda la región, impulsados por los crecientes costos de las materias primas.',
  'full_text': 'La inflación de marzo superó incluso los pronósticos más pesimistas en Brasil y Chile, según cifras oficiales publicadas el viernes. Eso sigue a datos de precios peores a los esperados en Colombia, Perú y México. La región está siendo golpeada por el aumento de los costos del combustible y los alimentos provocados por la invasión rusa a Ucrania. Las autoridades monetarias latinoameri

In [34]:
collection.insert_many(news)

> Empresas

In [35]:
client = MongoClient("localhost", 27017)
db = client["noticias"]
collection = db["empresas"]

In [36]:
news = []
url = '/archivo/economia/empresas/'

for n in range(1, 16):
    soup = make_request(browser, url + str(n))
    layout = soup.find(class_ = 'Layout-flexAds')
    blocks = layout.find('section').find_all(recursive = False)
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    for card in cards:
        news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [37]:
for i in news:
    soup = make_request(browser, i['relative_path'])
    
    i['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    i['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    i['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    paragraphs = soup.find_all(class_ = 'font--secondary')
    i['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [38]:
news

[{'title': 'El conglomerado de Warren Buffet compra acciones de HP por US$3.850 millones',
  'relative_path': '/economia/el-conglomerado-de-warren-buffet-compra-acciones-de-hp-por-us3850-millones/',
  'datetime': '8 abr 2022  - 2:17 p.\xa0m.',
  'author': 'EFE',
  'summary': 'Esta adquisición es la tercera operación que Berkshire cierra en un mes, después de que anunciara la compra de la aseguradora Alleghany por 11.600 millones de dólares.',
  'full_text': 'El conglomerado empresarial del veterano inversor Warren Buffet ha adquirido US$120,95 millones de acciones de la compañía tecnológica HP valoradas en más SU$4.200 millones. Según se recoge en los registros de la Comisión del Mercado de Valores de Estados Unidos (SEC), las compras de acciones han sido realizadas durante esta misma semana en varias operaciones distintas. Lea también: Oracle traslada su sede a Texas y se une a éxodo de California Los 120,95 millones de títulos que ha adquirido Berkshire Hathaway están valorados en al

In [39]:
collection.insert_many(news)